# This NoteBook is for the GEOLOCATION to generate CSVs from RP sessions based on predefined events 

In [1]:
import os
import sys
import pprint
import csv
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import radical.entk as re
import radical.utils     as ru
import radical.analytics as ra
import radical.pilot as rp
!radical-stack


/opt/anaconda2/lib/python2.7/site-packages/radical/pilot
/opt/anaconda2/lib/python2.7/site-packages/radical/utils
/opt/anaconda2/lib/python2.7/site-packages/radical/entk
/opt/anaconda2/lib/python2.7/site-packages/radical/saga
/opt/anaconda2/lib/python2.7/site-packages/radical/analytics

  python               : 2.7.14
  pythonpath           : 
  virtualenv           : /home/aymen/jupyter

  radical.analytics    : 0.70.0
  radical.entk         : 0.70.0
  radical.pilot        : 0.70.0
  radical.saga         : 0.70.0
  radical.utils        : 0.70.0



# This will not work with Design2 (only Design1)

In [5]:
loc = "/home/aymen/SummerRadical/paper_data/Design1/entk.session-design1-54875/"
src = os.path.dirname(loc)
sid = os.path.basename(loc)
session = ra.Session(sid=sid, stype='radical.pilot', src=loc)
event_entity = 'unit'
units = session.filter(etype=event_entity, inplace=True) 
dataframe=pd.DataFrame(columns=['ImageName1','ImageName2','ImageSize1','ImageSize2','TTX','UnitID'])
dataframe2=pd.DataFrame(columns=['ImageName1','ImageName2','ImageSize1','ImageSize2','TTX','UnitID'])
durations = []
for unit in units.get():
        
        if unit.uid == 'unit.000000': #skipping the first unit because unit.00000 is only for image discovery
            skip = True
            continue
            
        else:
            
            exec_duration = unit.duration(event=[{ru.EVENT: 'exec_start'},
                                                 {ru.EVENT: 'exec_stop'}])
            
            durations.append(exec_duration)
            cu_dur = sum(durations) / len(durations)
            
            myfilepath = '/home/aymen/SummerRadical/iceberg_escience/Geolocation/Data/Geolocation_Image_pairs.csv'
                            
            with open(myfilepath) as f:
                reader = csv.reader(f) 
                next(reader) # skip header
                data = []
                for row in reader:
                    data.append(row)

                    if (unit.description["name"][11:17]) == ("RANSAC"):

                        if (row[0] == unit.description["arguments"][2]) and row[1] == unit.description["arguments"][6]:



                            try:
                                  session_data = {
                                                        'ImageName1'      : row[0],
                                                        'ImageName2'      : row[1],
                                                        'ImageSize1'      : row[2],
                                                        'ImageSize2'      : row[3],
                                                        'TTX'             : exec_duration,
                                                        'UnitID'          : unit.uid[5:]
                                                }

                            except Exception as e:
                                                print e
                                                session_data = {
                                                         'ImageName'       : row[0],
                                                         'ImageName2'      : row[1],
                                                         'ImageSize'       : row[2],
                                                         'ImageSize2'      : row[3],
                                                         'TTX'             : exec_duration,
                                                         'UnitID'          : unit.uid[5:]
                                                               }


                            dataframe =  dataframe.append(session_data, ignore_index=True)
                            dataframe.to_csv('/home/aymen/SummerRadical/iceberg_escience/Geolocation/Data/units_durations_satge1.csv',index=False)
                            
                        else :
                            pass
                        
                    else:
                        
                        if (row[0] == unit.description["arguments"][0]) and row[1] == unit.description["arguments"][5]:
                            try:
                                      session_data = {
                                                            'ImageName1'      : row[0],
                                                            'ImageName2'      : row[1],
                                                            'ImageSize1'      : row[2],
                                                            'ImageSize2'      : row[3],
                                                            'TTX'             : exec_duration,
                                                            'UnitID'          : unit.uid[5:]
                                                    }

                            except Exception as e:
                                                    print e
                                                    session_data = {
                                                             'ImageName1'      : row[0],
                                                             'ImageName2'      : row[1],
                                                             'ImageSize1'      : row[2],
                                                             'ImageSize2'      : row[3],
                                                             'TTX'             : exec_duration,
                                                             'UnitID'          : unit.uid[5:]
                                                                   }


                            dataframe2 =  dataframe2.append(session_data, ignore_index=True)
                            dataframe2.to_csv('/home/aymen/SummerRadical/iceberg_escience/Geolocation/Data/units_durations_satge0.csv',index=False)
                        else:
                            pass
    

In [50]:
loc = "/home/aymen/SummerRadical/paper_data/Design1/entk.session-design1-54875/"
src = os.path.dirname(loc)
sid = os.path.basename(loc)
rp_session = ra.Session(stype='radical.pilot', src=src)# RP session
rp_pilots = rp_session.get(etype='pilot') 
rp_pilots = rp_session.filter(etype='pilot', inplace=False)
pilot_duration = rp_pilots.duration([rp.PMGR_ACTIVE, rp.FINAL])
rp_units = rp_session.filter(etype='unit', inplace=False)
durations = []
exec_duration = rp_units.duration(event=[{ru.EVENT: 'exec_start'}, {ru.EVENT: 'exec_stop'}])
durations.append(exec_duration)
cu_dur = sum(durations) / len(durations)
rp_dur = rp_units.duration([rp.NEW, rp.DONE])

In [48]:
for unit in rp_units.get():
    
    if (unit.description["name"][11:17]) == ("RANSAC"):
    
        print(unit.uid + ' is RANSAC and the TTC is : '+ str(unit.ttc))
    else :
        print(unit.uid + ' is Match and the TTC is : ' + str(unit.ttc))

NameError: name 'rp_units' is not defined

In [56]:
rp_units.get()[7].description['arguments'][5]

u'/pylon5/mc3bggp/aymen/geolocation_dataset_new/[CBAR]_WV03_20171214214134_1040010035CBB100_17DEC14214134-P1BS-501934391090_01_P001_u08rf3031.tif'

In [6]:
myfilepath = '/home/aymen/SummerRadical/paper_data/Design1/Geolocation_Image_pairs.csv'
with open(myfilepath) as f:
        reader = csv.reader(f) 
        next(reader) # skip header
        data = []
        for row in reader:
                    data.append(row)

In [6]:
row

,ImageName1,ImageName2,ImageSize1,ImageSize2,TTX,UnitID
